환경설정

In [12]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import tqdm
import jpype

In [9]:
folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_with_nouns.csv')
comments_df = pd.read_csv(file_path)

# 키워드 추출

In [ ]:

# 불용어 정의
stopwords = ['timecode', '진짜','이번','사람','정말', '뭔데', '그룹', '우리', '생각', '댓글', '느낌','계속', '지금', '최고', '영상', '처음', '축하', '대박', '이건', '당신', '제발', '항상', '아주', '다음', '정도', '모두', '보고', '그냥', '다시', '그것', '역시', '점점', '오늘', '요즘', '가장', '부분', '전부', '제일', '너머', '내용', '뭔가', '모습', '근데', '너무', '아니', '사람들', '같아요', '데리', '여러분', '세상', '자기', '다른']

# 고유명사 리스트 정의
custom_nouns = ['에스파', '조회수', '하이브', '어도어', 'BTS', '아이브', '르세라핌']

In [24]:
word_list = sample_df.iloc[0,:]['word_list']

In [57]:
import ast
from collections import defaultdict

def extract_keywords_frequency(series_word_list, top_n=100, min_df=1):
    # 시작시간 확인
    start_time = time.time()

    # 단어별 문서 빈도를 저장할 딕셔너리 (각 단어가 등장한 문서 수)
    doc_frequency = defaultdict(int)

    # 각 문서에서 등장한 단어 추출 및 빈도 계산
    for word_list in series_word_list:
        unique_words_in_doc = list(set(word_list))  # 문서 내에서 중복된 단어는 한 번만 카운트
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # min_df 조건을 충족하는 단어들의 빈도를 저장할 딕셔너리
    word_frequencies = defaultdict(int)

    # 각 문서의 단어 빈도 카운트 (min_df 조건에 맞는 단어만 빈도 계산)
    for word_list in series_word_list:
        for word in word_list:
            if doc_frequency[word] >= min_df:
                word_frequencies[word] += 1

    # 상위 빈도 단어 추출
    sorted_words = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)
    top_keywords_with_weights = sorted_words[:top_n]

    # 키워드만 추출
    top_keywords = [word for word, _ in top_keywords_with_weights]

    # 가중치를 전체 합으로 표준화
    total_count = sum(count for _, count in top_keywords_with_weights)
    keywords_with_weights = [(word, count / total_count) for word, count in top_keywords_with_weights]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [53]:
# TF-IDF 기반 키워드 및 상대적 비중 추출 함수
def extract_keywords_tfidf(series_word_list, top_n=100, min_df=1):

    # 시작시간 확인
    start_time = time.time()

    # 리스트 시리즈를 문자열 시리즈로 변형
    word_list_str = series_word_list.apply(lambda words: ' '.join(words))

    # TF-IDF 벡터화 (min_df를 설정해 빈도수가 적은 단어 제거)
    vectorizer = TfidfVectorizer(max_features=1000, min_df=min_df)
    tfidf_matrix = vectorizer.fit_transform(word_list_str)

    # 각 단어의 TF-IDF 점수를 추출
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = np.mean(tfidf_matrix.toarray(), axis=0)  # 전체 문서에서 평균 TF-IDF 추출

    # TF-IDF 점수를 기준으로 상위 키워드 추출
    top_n_indices = tfidf_scores.argsort()[-top_n:][::-1]
    top_keywords = [feature_names[idx] for idx in top_n_indices]

    # 각 단어의 TF-IDF 점수 추출
    weights = [tfidf_scores[idx] for idx in top_n_indices]

    # 가중치를 전체 합으로 표준화
    weights = weights / np.sum(weights)

    # 키워드와 가중치를 묶어서 반환
    keywords_with_weights = list(zip(top_keywords, weights))

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [50]:
from rank_bm25 import BM25Okapi
import time
import numpy as np
import pandas as pd
from collections import defaultdict

# BM25 기반 키워드 및 상대적 비중 추출 함수 (min_df 추가)
def extract_keywords_bm25(series_word_list, top_n=100, min_df=1):
    # 시작시간 확인
    start_time = time.time()

    # 시리즈의 각 리스트를 문서로 간주하여 BM25 모델 생성
    tokenized_corpus = series_word_list.tolist()
    bm25 = BM25Okapi(tokenized_corpus)

    # 각 단어의 출현 빈도를 저장할 딕셔너리
    doc_frequency = defaultdict(int)

    # 문서 내 각 단어의 출현 빈도 계산
    for doc in tokenized_corpus:
        unique_words_in_doc = set(doc)
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # 각 단어의 BM25 점수를 저장할 딕셔너리
    word_scores = defaultdict(float)
    total_docs = len(tokenized_corpus)

    # 각 문서에서의 BM25 점수를 계산하여 해당 단어에 대한 점수를 합산
    for doc in tokenized_corpus:
        scores = bm25.get_scores(doc)
        for word, score in zip(doc, scores):
            # 단어의 출현 빈도가 min_df 이상인 경우에만 점수 계산
            if doc_frequency[word] >= min_df:
                word_scores[word] += score

    # 문서 전체에 대한 평균 BM25 점수로 변환
    word_scores = {word: score / total_docs for word, score in word_scores.items()}

    # 상위 BM25 점수를 기준으로 키워드 추출
    sorted_words = sorted(word_scores.items(), key=lambda item: item[1], reverse=True)
    top_keywords = sorted_words[:top_n]

    # 가중치를 전체 합으로 표준화
    total_weight = sum(score for _, score in top_keywords)
    keywords_with_weights = [(word, score / total_weight) for word, score in top_keywords]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [60]:
sample_df = comments_df.copy()
#sample_df = comments_df.sample(frac=0.1, random_state=42)
print(len(sample_df), "\n")

# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 단순 빈도 기반 키워드 추출
print('**단순 빈도 기반 키워드 추출**\n')
keywords1, keywords_with_weights1 = extract_keywords_frequency(sample_df['word_list'], top_n=300, min_df=50)

# 결과 출력
for item in keywords_with_weights1: print(item)
print('\n')

# TF-IDF 기반 키워드 추출
print('**TF-IDF 기반 키워드 추출**\n')
keywords2, keywords_with_weights2 = extract_keywords_tfidf(sample_df['word_list'], top_n=300, min_df=50)

# 결과 출력
for item in keywords_with_weights2: print(item)
print('\n')

648060 

**단순 빈도 기반 키워드 추출**

648060개 댓글 키워드 추출에 걸린 시간: 1.1521368026733398 초

('노래', 0.06267088912923381)
('뉴진스', 0.04044226923645531)
('뮤비', 0.027513441592293745)
('아이브', 0.021102716329881185)
('하이브', 0.02050202693765124)
('사랑', 0.020448338377983023)
('민희진', 0.018436184533026246)
('단월드', 0.014549599670087692)
('컨셉', 0.010167368249052669)
('멤버', 0.0100413168480925)
('응원', 0.009197083699686427)
('에스파', 0.009012675168652106)
('사이비', 0.008673425719154366)
('화이팅', 0.008631408585500976)
('한국', 0.008589391451847587)
('가사', 0.007769279250538831)
('데뷔', 0.00753429454010691)
('아이돌', 0.007198935565947448)
('삭제', 0.007085333686069764)
('하나', 0.007005189894101261)
('중독', 0.006841011834825979)
('르세라핌', 0.006658937588994623)
('대표', 0.006640263307370895)
('여기', 0.0061531757950186354)
('방시혁', 0.006013896777908325)
('음악', 0.006001447256825839)
('앨범', 0.0058388253876858675)
('이제', 0.005617846388471744)
('리즈', 0.005595281631509738)
('파트', 0.005550152117585727)
('아이', 0.005508134983932337)
('컴백', 0.005340

In [68]:
# 출력 옵션 설정: 모든 열을 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_rows', None)     # 모든 행 출력

# 데이터프레임 전치 및 출력
compare_result  = pd.DataFrame([keywords1, keywords2]).transpose()
compare_result

,0,1
0,노래,노래
1,뉴진스,뉴진스
2,뮤비,아이브
3,아이브,사랑
4,하이브,뮤비
5,사랑,하이브
6,민희진,민희진
7,단월드,단월드
8,컨셉,화이팅
9,멤버,에스파


## 빈도기반 키워드 추출함수를 구현하는 두가지 방식의 속도 비교

In [ ]:
import time

# 빈도기반 키워드 추출함수 1 : scratch로 구현한 단순빈도기반 키워드 추출함수 정의
def extract_keywords_frequency1(series_word_list, top_n=100, min_df=1):
    # 시작시간 확인
    start_time = time.time()

    # 단어별 문서 빈도를 저장할 딕셔너리 (각 단어가 등장한 문서 수)
    doc_frequency = defaultdict(int)

    # 각 문서에서 등장한 단어 추출 및 빈도 계산
    for word_list in series_word_list:
        unique_words_in_doc = set(word_list)  # 문서 내에서 중복된 단어는 한 번만 카운트
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # min_df 조건을 충족하는 단어들의 빈도를 저장할 딕셔너리
    word_frequencies = defaultdict(int)

    # 각 문서의 단어 빈도 카운트 (min_df 조건에 맞는 단어만 빈도 계산)
    for word_list in series_word_list:
        for word in word_list:
            if doc_frequency[word] >= min_df:
                word_frequencies[word] += 1

    # 상위 빈도 단어 추출
    sorted_words = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)
    top_keywords = sorted_words[:top_n]

    # 가중치를 전체 합으로 표준화
    total_count = sum(count for _, count in top_keywords)
    keywords_with_weights = [(word, count / total_count) for word, count in top_keywords]

    # 종료시간 확인
    end_time = time.time()
    lap_time = end_time - start_time

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {lap_time} 초\n")

    return top_keywords, keywords_with_weights, lap_time

# 빈도기반 키워드 추출함수 2 : CountVectorizer를 이용한 단순빈도기반 키워드 추출함수 정의
def extract_keywords_frequency2(series_word_list, top_n=100, min_df=1):

    # 시작시간 확인
    start_time = time.time()

    # 리스트 시리즈를 문자열 시리즈로 변형
    word_list_str = series_word_list.apply(lambda words: ' '.join(words))

    # Count 벡터화 (단순 빈도 계산, min_df를 설정해 빈도수가 적은 단어 제거)
    vectorizer = CountVectorizer(max_features=1000, min_df=min_df)
    count_matrix = vectorizer.fit_transform(word_list_str)

    # 각 단어의 빈도수를 추출
    feature_names = vectorizer.get_feature_names_out()
    word_frequencies = np.sum(count_matrix.toarray(), axis=0)  # 전체 문서에서 빈도수 추출

    # 빈도수를 기준으로 상위 키워드 추출
    top_n_indices = word_frequencies.argsort()[-top_n:][::-1]
    top_keywords = [feature_names[idx] for idx in top_n_indices]

    # 각 단어의 빈도수 추출
    weights = [word_frequencies[idx] for idx in top_n_indices]

    # 가중치를 전체 합으로 표준화
    weights = weights / np.sum(weights)

    # 키워드와 가중치를 묶어서 반환
    keywords_with_weights = list(zip(top_keywords, weights))

    # 종료시간 확인
    end_time = time.time()
    lap_time = end_time - start_time

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {lap_time} 초\n")

    return top_keywords, keywords_with_weights, lap_time


# comment열에서 명사들이 들어있는 컬럼 생성
sample_df['word_list'] = sample_df['comment'].apply(extract_nouns)

# 빈도기반 키워드 추출함수 1
print('**단순 빈도 기반 키워드 추출 1**\n')
keywords, keywords_with_weights, time1 = extract_keywords_frequency1(sample_df['word_list'], top_n=10, min_df=5)

# 결과 출력
for item in keywords_with_weights: print(item)
print('\n')

# 빈도기반 키워드 추출함수 2
print('**단순 빈도 기반 키워드 추출 2**\n')
keywords, keywords_with_weights, time2 = extract_keywords_frequency2(sample_df['word_list'], top_n=10, min_df=5)

# 결과 출력
for item in keywords_with_weights: print(item)
print('\n')

print(f'function1(직접구현)이 function2(CountVectorizer)보다 {(time2/time1):.2f}배 더 빠릅니다')


**단순 빈도 기반 키워드 추출 1**

648개 댓글 키워드 추출에 걸린 시간: 0.001735687255859375 초

('노래', 0.2445054945054945)
('뉴진스', 0.17857142857142858)
('뮤비', 0.0989010989010989)
('하이브', 0.09615384615384616)
('민희진', 0.08516483516483517)
('아이브', 0.08516483516483517)
('사랑', 0.06593406593406594)
('단월드', 0.06043956043956044)
('멤버', 0.04395604395604396)
('응원', 0.04120879120879121)


**단순 빈도 기반 키워드 추출 2**

648개 댓글 키워드 추출에 걸린 시간: 0.013812541961669922 초

('노래', 0.2445054945054945)
('뉴진스', 0.17857142857142858)
('뮤비', 0.0989010989010989)
('하이브', 0.09615384615384616)
('민희진', 0.08516483516483517)
('아이브', 0.08516483516483517)
('사랑', 0.06593406593406594)
('단월드', 0.06043956043956044)
('멤버', 0.04395604395604396)
('응원', 0.04120879120879121)


function1(직접구현)이 function2(CountVectorizer)보다 7.96배 더 빠릅니다


# 특정 그룹/뮤직비디오에 대해 키워드 추출하기

In [ ]:
# MV가 뭐가 있더라
comments_df[comments_df['Group']=='NewJeans']['Title'].unique()

array(["'Hype Boy' Official MV (HANNI ver.)",
       "'Hype Boy' Official MV (MINJI ver.)",
       "'Hype Boy' Official MV (DANIELLE&HAERIN ver.)", 'Hurt',
       "'Hype Boy' Official MV (HYEIN ver.)", 'Cookie', 'Attention',
       'Zero', 'Supernatural (Part.2)', 'Supernatural (Part.1)',
       'Right Now', 'How Sweet', 'Bubble Gum', 'ASAP', 'ETA',
       "'Cool With You' & 'Get Up' (side B)", "'Cool With You' (side A)",
       'Super Shy', 'New Jeans', 'OMG', 'Ditto (side B)',
       'Ditto (side A)'], dtype=object)

In [ ]:
# How Sweet에 대한 댓글 키워드 분석
How_Sweet_df = comments_df[comments_df['Title']=='How Sweet']
How_Sweet_df = How_Sweet_df.sample(frac=0.1, random_state=1234)

# 분석할 댓글 건수 확인
len(How_Sweet_df)

3371

In [ ]:
# 'comment'에서 명사를 추출한 리스트를 'word_list'열에 저장
start_time = time.time()
How_Sweet_df['word_list'] = How_Sweet_df['comment'].apply(extract_nouns) #여기서 시간이 오래 걸림
end_time = time.time()
print(f"{len(sample_df)}개 댓글 명사 추출에 걸린 시간: {end_time - start_time:.2f} 초\n")

648개 댓글 명사 추출에 걸린 시간: 163.02 초



In [ ]:
# 단순 빈도 기반 키워드 추출
print('**단순 빈도 기반 키워드 추출**\n')
keywords, keywords_with_weights = extract_keywords_frequency(How_Sweet_df['word_list'], top_n=10, min_df=50)

# 결과 출력
for item in keywords_with_weights: print(item)
print('\n\n')

# TF-IDF 기반 키워드 추출
print('**TF-IDF 기반 키워드 추출**\n')
keywords, keywords_with_weights = extract_keywords_tfidf(How_Sweet_df['word_list'], top_n=10, min_df=50)

# 결과 출력
for item in keywords_with_weights: print(item)
print('\n\n')

# BM25 기반 키워드 추출
print('**BM25 기반 키워드 추출 (min_df 적용)**\n')
keywords, keywords_with_weights = extract_keywords_bm25(How_Sweet_df['word_list'], top_n=10, min_df=5)

# 결과 출력
for item in keywords_with_weights:
    print(item)

**단순 빈도 기반 키워드 추출**

3371개 댓글 키워드 추출에 걸린 시간: 0.009996414184570312 초

('뉴진스', 0.35336906584992345)
('하이브', 0.12672281776416539)
('뮤비', 0.0930321592649311)
('노래', 0.08996937212863707)
('민희진', 0.07312404287901991)
('제한', 0.06278713629402756)
('조회수', 0.06278713629402756)
('사랑', 0.054747320061255746)
('연령', 0.04326186830015314)
('응원', 0.04019908116385911)



**TF-IDF 기반 키워드 추출**

3371개 댓글 키워드 추출에 걸린 시간: 0.046727657318115234 초

('뉴진스', 0.32697438636712367)
('하이브', 0.12509912631892697)
('노래', 0.10345995387941447)
('뮤비', 0.09490976877623628)
('민희진', 0.07596239336180417)
('조회수', 0.06643167926333675)
('사랑', 0.06425586321232948)
('제한', 0.056458958735300414)
('응원', 0.04443465638181493)
('삭제', 0.04201321370371282)



**BM25 기반 키워드 추출 (min_df 적용)**

3371개 댓글 키워드 추출에 걸린 시간: 11.241628408432007 초

('뉴진스', 0.528653105564699)
('뮤비', 0.07217164727325977)
('사랑', 0.06535571679957808)
('하이브', 0.05980427251455962)
('노래', 0.054276157088470195)
('응원', 0.05055430059008709)
('민희진', 0.04631320915159922)
('뉴진스했다', 

In [ ]:
# "하이브"가 comment 컬럼에 포함된 행들을 필터링
filtered_df = How_Sweet_df[How_Sweet_df['comment'].str.contains('회수', na=False)]
sorted_filtered_df = filtered_df[['comment','date','likes']].sort_values(by='likes', ascending=False)
sorted_filtered_df[:10]

,comment,date,likes
298706,어쩐지 일요일 아침에도 500만? 600만 그렇게 찍힌거보고 의심 갔음.. 애초에 ...,2024-05-27T10:37:15Z,1692
296324,저도 증인입니다. 분명 830만 넘었었는데 820만대로 깍여져있네요. 진짜 하이브 ...,2024-05-27T22:14:59Z,1397
299766,연령제한 국가제한 뮤비 조회수 삭제 베댓 및 영어권 국가 팬들 댓글 삭제 스포지 신...,2024-05-27T04:45:33Z,1136
298776,연령 제한 걸고 지역 제한 걸고 댓글 삭제하고 조회수 없애고 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ㅋ...,2024-05-27T10:22:55Z,1079
285048,뉴진스 공식채널에서 올린음원이 하이브 공식 뮤비 보다 조회수 높은게 말이되나 이정도...,2024-06-04T07:05:16Z,580
298748,하이브야. 너네가 이럴수록.. 뉴진스 더 홍보 되어서 유튭 조회수 더 뛴다..,2024-05-27T10:28:25Z,523
296189,ㅅㅂ 첨에 공개하고 5시간 뒤에 14 482 000대 조회수 였는데 500만으로 떨...,2024-05-27T23:02:35Z,288
297186,방과 그 일당들은 상상조차 못할 미감이다.....옷도 어쩜 저리 이쁘게 입혔는지 너...,2024-05-27T15:21:21Z,174
300215,뮤비 조회수 삭제를 하이브가 어떻게 합니까… 하이브랑 별개로 이런 댓글 적으시면서 ...,2024-05-27T02:32:35Z,105
294479,노래가 너무 좋아서 나 하루에도 몇번씩이나 이 뮤비봄. 사람들이 제한걸려있다길래 아...,2024-05-28T09:29:25Z,92


In [ ]:
comment_sets = []
remaining_df = comments_df[comments_df['Title']=='How Sweet'].copy()  # 원본 데이터 손상을 방지하기 위해 복사

for i in range(5):
    sample_set = remaining_df.sample(frac=0.1, random_state=1234 + i)
    comment_sets.append(sample_set)
    remaining_df = remaining_df.drop(sample_set.index)  # 추출된 데이터를 원본에서 제거


In [ ]:
start = time.time()

# 단순 빈도 기반 키워드 추출
keywords_list = []

for comment_set in comment_sets:
    keywords, keywords_with_weights = extract_keywords_frequency(comment_set['comment'].apply(extract_nouns), top_n=10, min_df=5)
    keywords_list.append(keywords)

end = time.time()
print(f"\nExecution time: {end - start} seconds")

pd.DataFrame(keywords_list).transpose()

3371개 댓글 키워드 추출에 걸린 시간: 0.009246349334716797 초

3034개 댓글 키워드 추출에 걸린 시간: 0.008253097534179688 초

2730개 댓글 키워드 추출에 걸린 시간: 0.007547616958618164 초

2457개 댓글 키워드 추출에 걸린 시간: 0.008022069931030273 초

2212개 댓글 키워드 추출에 걸린 시간: 0.00545048713684082 초


Execution time: 395.77906584739685 seconds


,0,1,2,3,4
0,"(뉴진스, 923)","(뉴진스, 878)","(뉴진스, 794)","(뉴진스, 661)","(뉴진스, 599)"
1,"(하이브, 331)","(하이브, 298)","(하이브, 277)","(하이브, 245)","(하이브, 212)"
2,"(뮤비, 243)","(뮤비, 215)","(뮤비, 207)","(노래, 165)","(뮤비, 185)"
3,"(노래, 235)","(노래, 215)","(노래, 190)","(뮤비, 161)","(노래, 157)"
4,"(민희진, 191)","(제한, 192)","(제한, 155)","(민희진, 133)","(제한, 97)"
5,"(제한, 164)","(민희진, 176)","(민희진, 145)","(제한, 124)","(민희진, 91)"
6,"(조회수, 164)","(조회수, 129)","(조회수, 138)","(조회수, 109)","(조회수, 85)"
7,"(사랑, 143)","(연령, 127)","(사랑, 112)","(사랑, 93)","(사랑, 79)"
8,"(연령, 113)","(사랑, 119)","(연령, 104)","(연령, 80)","(연령, 69)"
9,"(응원, 105)","(응원, 109)","(응원, 94)","(응원, 74)","(응원, 59)"


In [ ]:
import concurrent.futures
import pandas as pd

start = time.time()

# 단순 빈도 기반 키워드 추출
keywords_list = []

# 각 comment_set을 처리할 함수 정의
def process_comment_set(comment_set):
    # 샘플링된 데이터에서 키워드 추출
    keywords, keywords_with_weights = extract_keywords_frequency(
        comment_set['comment'].sample(frac=0.1).apply(extract_nouns),
        top_n=10,
        min_df=5
    )
    return keywords

# 쓰레딩을 사용하여 병렬로 처리
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 각 comment_set을 쓰레딩을 통해 병렬 처리
    results = list(executor.map(process_comment_set, comment_sets))

# 결과를 keywords_list에 추가
keywords_list.extend(results)

end = time.time()
print(f"\nExecution time: {end - start} seconds")

pd.DataFrame(keywords_list).transpose()


221개 댓글 키워드 추출에 걸린 시간: 0.0009396076202392578 초

273개 댓글 키워드 추출에 걸린 시간: 0.0007424354553222656 초

303개 댓글 키워드 추출에 걸린 시간: 0.0008327960968017578 초

337개 댓글 키워드 추출에 걸린 시간: 0.0010840892791748047 초

246개 댓글 키워드 추출에 걸린 시간: 0.0007205009460449219 초


Execution time: 13.128170251846313 seconds


,0,1,2,3,4
0,"(뉴진스, 97)","(뉴진스, 83)","(뉴진스, 80)","(뉴진스, 71)","(뉴진스, 61)"
1,"(하이브, 35)","(하이브, 30)","(하이브, 29)","(하이브, 24)","(뮤비, 25)"
2,"(뮤비, 31)","(제한, 23)","(노래, 24)","(노래, 16)","(하이브, 24)"
3,"(조회수, 29)","(노래, 19)","(민희진, 19)","(조회수, 12)","(노래, 15)"
4,"(노래, 23)","(뮤비, 18)","(뮤비, 19)","(민희진, 12)","(제한, 11)"
5,"(민희진, 23)","(연령, 16)","(응원, 16)","(뮤비, 11)","(사랑, 9)"
6,"(제한, 20)","(민희진, 15)","(사랑, 12)","(제한, 11)","(삭제, 8)"
7,"(연령, 16)","(사랑, 14)","(제한, 12)","(사랑, 10)","(조회수, 8)"
8,"(삭제, 15)","(조회수, 12)","(조회수, 11)","(음악, 10)","(연령, 7)"
9,"(버니, 11)","(응원, 11)","(음악, 11)","(방시혁, 7)","(이제, 7)"
